# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [11]:
#Loading the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('homepage_actions.csv')
df.head(10)

/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [25]:
experiment = set(df[df['group'] == 'experiment']['id'])
control = set(df[df['group'] == 'control']['id'])
print("Number of Id's that are in both groups:",len(experiment & control))

Number of Id's that are in both groups: 0


In [24]:
click = set(df[df['action'] == 'click']['id'].unique())
view = set(df[df['action'] == 'view']['id'].unique())
print('Number of people who viewed:', len(view))
print('Number of people who clicked:', len(click))
print("Number of people who clicked but didn't view:", len(click - view))

Number of people who viewed: 6328
Number of people who clicked: 1860
Number of people who clicked but didn't view: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [37]:
#Went a different route but this added a one for each successful click
i=0
df['Worked'] = 0
for x in df['action']:
    if x == 'click':
        df['Worked'][i] = 1
    else:
        df['Worked'][i] = 0
    i+=1
df.head(10)

/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,timestamp,id,group,action,Worked
0,2016-09-24 17:42:27.839496,804196,experiment,view,0
1,2016-09-24 19:19:03.542569,434745,experiment,view,0
2,2016-09-24 19:36:00.944135,507599,experiment,view,0
3,2016-09-24 19:59:02.646620,671993,control,view,0
4,2016-09-24 20:26:14.466886,536734,experiment,view,0
5,2016-09-24 20:32:25.712659,681598,experiment,view,0
6,2016-09-24 20:39:03.248853,522116,experiment,view,0
7,2016-09-24 20:57:20.336757,349125,experiment,view,0
8,2016-09-24 20:58:01.948663,349125,experiment,click,1
9,2016-09-24 21:00:12.278374,560027,control,view,0


In [60]:
df['Count'] = 1

experiment_data = df[df['group'] == 'experiment']
control_data = df[df['group'] == 'control']
experiment_data = experiment_data.pivot(index='id',columns='action',values='Count')
experiment_data = experiment_data.fillna(value=0)
control_data = control_data.pivot(index='id',columns='action',values='Count')
control_data = control_data.fillna(value=0)

print('Percentage of clicks for experiment:',experiment_data.click.mean())
print('Percentage of clicks for control:',control_data.click.mean())


Percentage of clicks for experiment: 0.3097463284379172
Percentage of clicks for control: 0.2797118847539016


In [71]:
import flatiron_stats as fs

fs.p_value_welch_ttest(control_data.click,experiment_data.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [64]:
expected = len(experiment_data)*(control_data.click.mean())
expected

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [66]:
n = len(experiment_data)
p = control_data.click.mean()
var = n*p*(1-p)
std = np.sqrt(var)
std

24.568547907005815

In [68]:
difference = (experiment_data.click.sum()-expected)/std
difference

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [70]:
import scipy.stats as stats
p_val = stats.norm.sf(difference)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p_value for verifying results is significantly better but they are both below the alpha value
so the H0 would be rejected in both cases.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.